In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import pymongo
import xarray as xr

In [2]:
def format(LON):
    if LON < 0:
        LON = LON + 360
    else:
        LON = LON
    return LON

In [3]:
station = 0
overlap = 0
result = []

In [4]:
rootdir = '/home/cz2397/data/cchdo-ctd/'

In [5]:
client = MongoClient()
db = client.eddies
rcs_cchdo = db.rcs_cchdo

In [6]:
rcs_cchdo.create_index([('dat', pymongo.ASCENDING)])

u'dat_1'

In [7]:
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        if file.endswith('ctd.nc'):
            path = os.path.join(subdir, file)
            DS = xr.open_dataset(path)
            DAT = DS.time.values[0]
            LAT = float(DS.latitude.values[0])
            LON = float(format(DS.longitude.values[0]))
            dat_var = np.timedelta64(4, 'D')
            dat_min = pd.to_datetime(DAT - dat_var)
            dat_max = pd.to_datetime(DAT + dat_var)
            ft = {
                'dat': {'$gt': dat_min, '$lt': dat_max},
                'loc': {'$within': {'$center': [[LON, LAT], 1]}}
            }
            count = rcs_cchdo.find(ft).count()
            station = station + 1
            if count != 0:
                overlap = overlap + 1
                result.append(path)

100%|██████████| 115/115 [00:01<00:00, 73.59it/s]


In [8]:
station

33603

In [9]:
overlap

851

In [10]:
result.sort()

In [11]:
result

['/home/cz2397/data/cchdo-ctd/106_33KI160_1_nc_ctd/prs2_00001_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/106_33KI160_1_nc_ctd/prs2_00006_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/125_pr06_c_nc_ctd/pr06_c_prs1_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/125_pr06_c_nc_ctd/pr06_c_prs1_00004_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00002_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00003_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00004_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00005_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00006_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00007_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00008_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/12_33KI121_1_nc_ctd/prs2_00002_00009_ctd.nc',
 '/home/cz23